## Chains
链是LangChain的核心构建模块，通常将大型语言模型（LLM）和提示（Prompt）结合在一起。


## LLMChain
最简单的Chain类型，直接使用即可

In [17]:

from langchain import PromptTemplate, OpenAI

template = """\
你是一个新公司的命名咨询顾问.
为制作 {product} 的公司起好的名字? 使用中文回答问题,不少于5个名字
"""

prompt = PromptTemplate.from_template(template)
# prompt.format(product="五颜六色的袜子")
llm = OpenAI(temperature=0.9)

In [18]:
from langchain.chains import LLMChain
# Chain可以把llm和Prompt组合在一起
chain = LLMChain(llm=llm, prompt=prompt,verbose=True)
print(chain.run("五颜六色的袜子"))



> Entering new LLMChain chain...
Prompt after formatting:
你是一个新公司的命名咨询顾问.
为制作 五颜六色的袜子 的公司起好的名字? 使用中文回答问题,不少于5个名字


> Finished chain.

1. 色彩逸品 
2. 五色薰心 
3. 缤纷飨宴 
4. 鸿运袜家 
5. 足下时尚


## SimpleSequentialChain
每个步骤都有一个单一的输入/输出，一个步骤的输出是下一个步骤的输入。

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import SimpleSequentialChain

llm = ChatOpenAI(temperature=0.9)
# 第一个Prompt和Chain
first_prompt = ChatPromptTemplate.from_template(
    "你是一个新公司的命名咨询顾问.为制作 {product} 的公司起一个好的名字? 使用中文回答问题"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# 第二个Prompt和Chain
second_prompt = ChatPromptTemplate.from_template(
    "为下面的公司写一个20字的简短描述：{company_name}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [6]:
# 把第一个Chain和第二个Chain合在一起
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
overall_simple_chain.run("五颜六色的袜子")



> Entering new SimpleSequentialChain chain...
彩虹丝袜公司
彩虹丝袜公司：提供丰富多彩的时尚丝袜，让你的腿部焕发绚丽色彩。

> Finished chain.


'彩虹丝袜公司：提供丰富多彩的时尚丝袜，让你的腿部焕发绚丽色彩。'

## Sequential Chains
不是所有的链都是有固定的输入和输出，有时候中间的链需要多个输入，最终也有多个输出，这个时候考虑用SequentialChain

In [7]:
# 这是一个LLMChain，给定一个剧本的标题和它所处的时代，它的任务是写一个概要。
llm = OpenAI(temperature=.7)
template = """你是一位剧作家。给定剧本的标题和它所处的时代，你的任务是为该标题写一个概要。

标题: {title}
时代: {era}
剧作家: 这是上述剧本的概要:"""
prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis")

# 这是一个LLMChain，给定一个剧本的概要，它的任务是写一个剧本的评论。
llm = OpenAI(temperature=.7)
template = """你是一位专业的剧本评论家。给定剧本的概要，你的任务是为该剧本写一篇评论。

剧本概要:
{synopsis}
你对上述剧本的评论:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")


# 这是整体链，我们按顺序运行这两个链。
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # 这里我们返回多个变量
    output_variables=["synopsis", "review"],
    verbose=True)

overall_chain({"title":"海滩上的日落悲剧", "era": "维多利亚时代的英格兰"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': '海滩上的日落悲剧',
 'era': '维多利亚时代的英格兰',
 'synopsis': ' “海滩上的日落悲剧”是一部充满爱情、矛盾和复仇的维多利亚时代的英格兰悲剧。它讲述了一位女士，她的丈夫意外地死去，并留下了一笔遗产给她。她决定将遗产捐给一个有悲剧背景的家庭。然而，这桩善行引起了一位男士的愤怒，他决定为此复仇，最终导致了一个悲惨的结局。',
 'review': '\n\n《海滩上的日落悲剧》是一部充满爱情、矛盾和复仇的维多利亚时代英格兰悲剧。它讲述了一位女士，她的丈夫突然去世，留给她一笔遗产。她决定将这笔遗产捐给一个具有悲剧背景的家庭，而这一善行却引起了一位男士的愤怒，他决定复仇，最终导致一个悲惨的结局。\n\n这部剧'}

## RouterChains
有时候单个串行的Chain不能满足我们的诉求，这个时候考虑使用RouterChain
它在一系列的链（Chain）中动态地选择下一个要执行的链。这种模式通常用于处理复杂的逻辑流程，其中下一个执行的步骤取决于当前的输入或状态。


In [8]:
#例如，如果你正在构建一个问题回答系统，你可能有多个链，每个链专门处理一种类型的问题
# （例如，一个处理物理问题，一个处理数学问题等）。
# 然后，你可以使用一个"RouterChain"来检查每个问题的特性，并将问题路由到最适合处理该问题的链。
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

physics_template = """你是一位非常聪明的物理教授。 \
你擅长以简洁易懂的方式回答物理问题。 \
当你不知道问题的答案时，你会承认你不知道。

这是一个问题：
{input}"""

math_template = """你是一位非常好的数学家。你擅长回答数学问题。 \
你之所以这么好，是因为你能够将难题分解成各个组成部分， \
回答组成部分，然后将它们组合起来回答更广泛的问题。

这是一个问题：
{input}"""

prompt_infos = [
    {  "name": "物理", "description": "适合回答物理问题","prompt_template": physics_template,},
    {  "name": "数学", "description": "适合回答数学问题","prompt_template": math_template,},
]

llm = OpenAI()

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

# 默认的Chain
default_chain = ConversationChain(llm=llm, output_key="text")


In [13]:
# import pprint
# pprint.pprint(destination_chains)
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
print(destination_chains.keys())
print(destinations)

dict_keys(['物理', '数学'])
['物理: 适合回答物理问题', '数学: 适合回答数学问题']


In [14]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
# 物理: 适合回答物理问题', '数学: 适合回答数学问题
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_prompt_template = """\
给定一个原始的文本输入到语言模型中，选择最适合输入的模型提示。
你将得到可用提示的名称和提示最适合的描述。如果你认为修改原始输入最终会得到更好的语言模型响应，你也可以修改原始输入。

<< 格式化 >>
返回一个markdown代码片段，其中包含一个格式化为如下样式的JSON对象：
```json
{{{{
    "destination": string \\ 使用的提示名称或"DEFAULT"
    "next_inputs": string \\ 可能修改过的原始输入
}}}}
```

记住："destination" 必须是下面指定的候选提示名称之一，或者如果输入不适合任何候选提示，它可以是"DEFAULT"。
记住："next_inputs" 可以是原始输入，如果你认为不需要任何修改。

<< 候选提示 >>
{destinations}

<< 输入 >>
{{input}}

<< 输出 >>
"""
router_template = router_prompt_template.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# 构建RouterChains
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [15]:
print(chain.run("什么是黑体辐射?"))



> Entering new MultiPromptChain chain...


/Users/aihe/PycharmProjects/langchan_tutorial/venv/lib/python3.9/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


物理: {'input': '什么是黑体辐射?'}
> Finished chain.


黑体辐射是指热物体，如火或太阳，发出的电磁波辐射。它的特征是温度越高，发出的电磁波越多，发出的频率也越高。


In [18]:
print(chain.run("计算下7乘以24，然后再乘以60等于多少？"))



> Entering new MultiPromptChain chain...
数学: {'input': '计算 7 乘以 24 乘以 60 等于多少？'}
> Finished chain.


答案：7 乘以 24 乘以 60 等于 8,640。


In [19]:
print(chain.run("什么是彩虹？"))



> Entering new MultiPromptChain chain...
None: {'input': '什么是彩虹？'}
> Finished chain.
 彩虹是一种美丽的天空现象，由多种颜色的光线混合而成。它由水滴和冰晶反射出来的多种颜色的光线组成，这些颜色从红色到紫色排列。
